In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

from dotenv import load_dotenv
import pandas as pd
import os
import requests

load_dotenv()

True

In [2]:
credentials_dict = {
    "type": os.getenv("BIG_QUERY_TYPE"),
    "project_id": os.getenv("BIG_QUERY_PROJECT_ID"),
    "private_key_id": "dummy_key_id",  # 없어도 무방
    "private_key": os.getenv("BIG_QUERY_PRIVATE_KEY").replace("\\n", "\n"),
    "client_email": os.getenv("BIG_QUERY_CLIENT_EMAIL"),
    "client_id": os.getenv("BIG_QUERY_CLIENT_ID"),
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": f"https://www.googleapis.com/robot/v1/metadata/x509/{os.getenv('BIG_QUERY_CLIENT_EMAIL').replace('@', '%40')}",
    "universe_domain": os.getenv("BIG_QUERY_UNIVERSE_DOMAIN", "googleapis.com")
}

credentials = service_account.Credentials.from_service_account_info(credentials_dict)


In [3]:
project_id = os.getenv("BIG_QUERY_PROJECT_ID")
client = bigquery.Client(credentials=credentials, project=project_id)
client.project

'sixth-topic-349709'

In [4]:
query = f"""
    SELECT *
    FROM `{client.project}.ga4_events_partitioned.events`
    order by partition_date DESC
    LIMIT 5
    """

res = client.query(query).result()

for i in res:
    print(i)

Row(('20250623', 1750648540698587, 'scroll_to_50', [{'key': 'engagement_time_msec', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}, {'key': 'page_referrer', 'value': {'string_value': 'https://hanpoom.com/account/login', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'batch_page_id', 'value': {'string_value': None, 'int_value': 1750648532901, 'float_value': None, 'double_value': None}}, {'key': 'engaged_session_event', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}, {'key': 'page_title', 'value': {'string_value': 'Login | Hanpoom | 한품 | 한국을 내품에 미국에서 한국 쇼핑은 한품', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'ga_session_id', 'value': {'string_value': None, 'int_value': 1750648526, 'float_value': None, 'double_value': None}}, {'key': 'batch_ordering_id', 'value': {'string_value': None, 'int_value': 4, 'float_value': None, 'double_value': None}}, {'key

In [17]:
def get_table_schema_info(client, dataset_id, table_id):
    """테이블 스키마 정보 조회"""
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)
    
    # print(f"테이블: {table.project}.{table.dataset_id}.{table.table_id}")
    # print(f"생성일: {table.created}")
    # print(f"총 행 수: {table.num_rows:,}")
    # print(f"테이블 크기: {table.num_bytes:,} bytes")
    # print("\n컬럼 정보:")
    
    columns_info = []
    for field in table.schema:
        column_info = {
            'column_name': field.name,
            'data_type': field.field_type,
            'mode': field.mode,
            'description': field.description or ''
        }
        columns_info.append(column_info)
        # print(f"  - {field.name}: {field.field_type} ({field.mode})")
    
    return columns_info

def query_table_to_dataframe(client, dataset_id, table_id, limit=10, order_by=None):
    """테이블 데이터를 조회하여 DataFrame으로 변환"""
    # 테이블 스키마 정보 먼저 확인
    schema_info = get_table_schema_info(client, dataset_id, table_id)
    
    # 컬럼명 리스트 생성
    column_names = [col['column_name'] for col in schema_info]
    # print(f"\n컬럼 목록: {column_names}")
    
    # 쿼리 작성
    order_clause = f"ORDER BY {order_by}" if order_by else ""
    query = f"""
    SELECT *
    FROM `{client.project}.{dataset_id}.{table_id}`
    {order_clause}
    LIMIT {limit}
    """
    
    try:
        print(f"\n실행 쿼리:\n{query}")
        query_result = client.query(query).result()
        
        # DataFrame으로 변환
        df = query_result.to_dataframe()
        
        return df
        
    except Exception as e:
        print(f"테이블 조회 실패: {e}")
        return None, column_names, schema_info

In [18]:
query_table_to_dataframe(client, "ga4_events_partitioned", "events", limit=5, order_by="partition_date DESC")


실행 쿼리:

    SELECT *
    FROM `sixth-topic-349709.ga4_events_partitioned.events`
    ORDER BY partition_date DESC
    LIMIT 5
    


/Users/robert/Desktop/airflow_study/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,ecommerce,items,collected_traffic_source,is_active_user,batch_event_index,batch_page_id,batch_ordering_id,session_traffic_source_last_click,publisher,partition_date
0,20250623,1750648540698587,scroll_to_50,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-1603987493,<NA>,75266,405453324.1729091460,...,None,[],None,True,5,1750648532901,4,"{'manual_campaign': {'campaign_id': None, 'cam...",None,2025-06-23
1,20250623,1750675811616438,page_view,"[{'key': 'page_referrer', 'value': {'string_va...",<NA>,NaN,-102873418,<NA>,None,405453324.1729091460,...,None,[],"{'manual_campaign_id': None, 'manual_campaign_...",True,2,1750675810523,1,"{'manual_campaign': {'campaign_id': None, 'cam...",None,2025-06-23
2,20250623,1750675821434463,scroll_to_50,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-93055393,<NA>,75266,405453324.1729091460,...,None,[],None,True,1,1750675810523,3,"{'manual_campaign': {'campaign_id': None, 'cam...",None,2025-06-23
3,20250623,1750627033789403,virtual_page_view,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,-1636060197,<NA>,94641,1283247621.1730387273,...,None,[],None,True,1,1750626830102,19,"{'manual_campaign': {'campaign_id': None, 'cam...",None,2025-06-23
4,20250623,1750683869302807,scroll_to_25,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,-635121641,<NA>,75266,405453324.1729091460,...,None,[],None,True,2,1750683865223,3,"{'manual_campaign': {'campaign_id': None, 'cam...",None,2025-06-23


In [24]:

def extract(url):
    f = requests.get(url) 
    return (f.text) 

def transform(text):
    lines = text.strip().split("\n") 
    records = [] 
    for l in lines:
        (country, capital) = l.split(",")
        records.append([country, capital])
    return records[1:]

In [35]:
from google.api_core.exceptions import NotFound

def load_bq(client, records, dataset_id="raw_data", table_id="country_capital"):
    """
    BigQuery에 레코드 업로드
    :param client: bigquery.Client
    :param records: [(country, capital), ...]
    """
    # 1. 테이블 이름 설정
    table_ref = f"{client.project}.{dataset_id}.{table_id}"

    try:
        client.get_dataset(dataset_id)
    except NotFound:
        dataset = bigquery.Dataset(f"{client.project}.{dataset_id}")
        dataset.location = "US"  # 또는 asia-northeast3 등 사용 중인 지역
        client.create_dataset(dataset)
        print(f"📁 Dataset `{dataset_id}` created.")

    # 2. 스키마 정의 (생략하면 자동 추론 가능)
    schema = [
        bigquery.SchemaField("country", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("capital", "STRING", mode="NULLABLE"),
    ]

    # 3. 데이터프레임으로 변환
    df = pd.DataFrame(records, columns=["country", "capital"])

    # 4. 테이블이 없으면 생성
    try:
        client.get_table(table_ref)
    except Exception:
        table = bigquery.Table(table_ref, schema=schema)
        client.create_table(table)

    # 5. 데이터 업로드 (기존 데이터에 추가)
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()  # 완료까지 대기

    print(f"✅ {len(df)} rows loaded into {table_ref}")

In [36]:
link = "https://s3-geospatial.s3.us-west-2.amazonaws.com/country_capital.csv"
data = extract(link)
data

"country,capital\nAbkhazia,Sukhumi\nAfghanistan,Kabul\nAkrotiri and Dhekelia,Episkopi Cantonment\nAlbania,Tirana\nAlgeria,Algiers\nAmerican Samoa,Pago Pago\nAndorra,Andorra la Vella\nAngola,Luanda\nAnguilla,The Valley\nAntigua and Barbuda,St. John's\nArgentina,Buenos Aires\nArmenia,Yerevan\nAruba,Oranjestad\nAscension Island,Georgetown\nAustralia,Canberra\nAustria,Vienna\nAzerbaijan,Baku\nBahamas,Nassau\nBahrain,Manama\nBangladesh,Dhaka\nBarbados,Bridgetown\nBelarus,Minsk\nBelgium,Brussels\nBelize,Belmopan\nBenin,Porto-Novo\nBermuda,Hamilton\nBhutan,Thimphu\nBolivia,La Paz\nBosnia and Herzegovina,Sarajevo\nBotswana,Gaborone\nBrazil,Brasâ\x88\x9aâ\x89\xa0lia\nBritish Virgin Islands,Road Town\nBrunei,Bandar Seri Begawan\nBulgaria,Sofia\nBurkina Faso,Ouagadougou\nBurundi,Bujumbura\nCambodia,Phnom Penh\nCameroon,Yaoundâ\x88\x9aÂ©\nCanada,Ottawa\nCape Verde,Praia\nCayman Islands,George Town\nCentral African Republic,Bangui\nChad,N'Djamena\nChile,Santiago\nChina,Beijing\nChristmas Island,Fly

In [37]:
records = transform(data)
records

[['Abkhazia', 'Sukhumi'],
 ['Afghanistan', 'Kabul'],
 ['Akrotiri and Dhekelia', 'Episkopi Cantonment'],
 ['Albania', 'Tirana'],
 ['Algeria', 'Algiers'],
 ['American Samoa', 'Pago Pago'],
 ['Andorra', 'Andorra la Vella'],
 ['Angola', 'Luanda'],
 ['Anguilla', 'The Valley'],
 ['Antigua and Barbuda', "St. John's"],
 ['Argentina', 'Buenos Aires'],
 ['Armenia', 'Yerevan'],
 ['Aruba', 'Oranjestad'],
 ['Ascension Island', 'Georgetown'],
 ['Australia', 'Canberra'],
 ['Austria', 'Vienna'],
 ['Azerbaijan', 'Baku'],
 ['Bahamas', 'Nassau'],
 ['Bahrain', 'Manama'],
 ['Bangladesh', 'Dhaka'],
 ['Barbados', 'Bridgetown'],
 ['Belarus', 'Minsk'],
 ['Belgium', 'Brussels'],
 ['Belize', 'Belmopan'],
 ['Benin', 'Porto-Novo'],
 ['Bermuda', 'Hamilton'],
 ['Bhutan', 'Thimphu'],
 ['Bolivia', 'La Paz'],
 ['Bosnia and Herzegovina', 'Sarajevo'],
 ['Botswana', 'Gaborone'],
 ['Brazil', 'Brasâ\x88\x9aâ\x89\xa0lia'],
 ['British Virgin Islands', 'Road Town'],
 ['Brunei', 'Bandar Seri Begawan'],
 ['Bulgaria', 'Sofia'],
 

In [41]:
load_bq(client, records)

/Users/robert/Desktop/airflow_study/.venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ 247 rows loaded into sixth-topic-349709.raw_data.country_capital


In [42]:
query_table_to_dataframe(client, "raw_data", "country_capital", limit=10, order_by='country')


실행 쿼리:

    SELECT *
    FROM `sixth-topic-349709.raw_data.country_capital`
    ORDER BY country
    LIMIT 10
    


/Users/robert/Desktop/airflow_study/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,country,capital
0,Abkhazia,Sukhumi
1,Abkhazia,Sukhumi
2,Afghanistan,Kabul
3,Afghanistan,Kabul
4,Akrotiri and Dhekelia,Episkopi Cantonment
5,Akrotiri and Dhekelia,Episkopi Cantonment
6,Albania,Tirana
7,Albania,Tirana
8,Algeria,Algiers
9,Algeria,Algiers


In [47]:
def load_full_refresh(client, records, dataset_id="raw_data", table_id="country_capital"):
    table_ref = f"{client.project}.{dataset_id}.{table_id}"

    # 1. Dataset이 없으면 생성
    try:
        client.get_dataset(dataset_id)
    except NotFound:
        dataset = bigquery.Dataset(f"{client.project}.{dataset_id}")
        dataset.location = "US"
        client.create_dataset(dataset)
        print(f"📁 Dataset `{dataset_id}` created.")

    # 2. DataFrame 준비
    df = pd.DataFrame(records, columns=["country", "capital"])

    # 테이블 존재 여부 확인
    try:
        client.get_table(table_ref)
        print(f"🧾 Table `{table_id}` already exists, will overwrite it.")
    except NotFound:
        # 처음 한 번만 스키마 지정해서 테이블 생성
        schema = [
            bigquery.SchemaField("country", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("capital", "STRING", mode="NULLABLE"),
        ]
        table = bigquery.Table(table_ref, schema=schema)
        client.create_table(table)
        print(f"🧾 Table `{table_id}` created.")


    # 5. 테이블 덮어쓰기 (Full Refresh 핵심)
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")  # 기존 데이터 삭제 후 덮어쓰기
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()

    print(f"✅ {len(df)} rows fully refreshed into `{table_ref}`")

In [48]:
load_full_refresh(client, records)

🧾 Table `country_capital` already exists, will overwrite it.


/Users/robert/Desktop/airflow_study/.venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ 247 rows fully refreshed into `sixth-topic-349709.raw_data.country_capital`


In [49]:
query_table_to_dataframe(client, "raw_data", "country_capital", limit=10, order_by='country')


실행 쿼리:

    SELECT *
    FROM `sixth-topic-349709.raw_data.country_capital`
    ORDER BY country
    LIMIT 10
    


/Users/robert/Desktop/airflow_study/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,country,capital
0,Abkhazia,Sukhumi
1,Afghanistan,Kabul
2,Akrotiri and Dhekelia,Episkopi Cantonment
3,Albania,Tirana
4,Algeria,Algiers
5,American Samoa,Pago Pago
6,Andorra,Andorra la Vella
7,Angola,Luanda
8,Anguilla,The Valley
9,Antigua and Barbuda,St. John's
